# FRP fixation-based statistics

## Import libraries

In [1]:
import numpy as np
from tqdm.notebook import tqdm
from utils.utils import *
from mne.stats import *
import mne

import scipy
mne.set_log_level('CRITICAL')

In [2]:
behavioral_data = pd.read_csv(get_behavioral_data_path(final_data_exclusion=True), sep=SEPARATOR, index_col=False, dtype={PARTICIPANT: str})
special_fixation_data = pd.read_csv(get_special_fixations_path(), sep=SEPARATOR, index_col=False, dtype={PARTICIPANT: str})
included_trials_path = get_all_erp_fixation_analysis_paths(False, 'erp frp offset')
for path in tqdm(included_trials_path):
    print(path)
    included_trials = pd.read_csv(path, sep=SEPARATOR, index_col=False, dtype={PARTICIPANT: str})
    epoch_interval=[int(t) for t in path.stem.split('_')[1:3]]
    frp=[fs_long for fs_long, fs_short in FIXATION_SELECTION_SHORT_VERSION.items() if fs_short == path.parts[3].split('_')[2]][0]
    try:
        snippet_group = path.parts[3].split('_')[3]
    except IndexError:
        snippet_group = SNIPPET_GROUP_ALL
    description = get_erp_description(frp, False, [e/1000 for e in epoch_interval])

    data = pd.DataFrame(columns=[PATH, ERP_PARAMETER_EPOCH_INTERVAL, SITUATION, ERP_PARAMETER_ERP_FRP, PARTICIPANT, CONDITION, 
                                 'Offset from stimulus onset to fixation onset', 'Fixation Duration','Duration to next fixation', 'Offset to First Fixation'])
    for i, row in included_trials.iterrows():

        participant, snippet=row[PARTICIPANT], row[SNIPPET]
        # print(participant, snippet)
        # identify corresponding fixation
        special_fixation = special_fixation_data[(special_fixation_data[PARTICIPANT]==participant) & (special_fixation_data[SNIPPET]==snippet) & (special_fixation_data[FIXATION_SELECTION_ALGORITHM]==frp)].squeeze()
        behavior = behavioral_data[(behavioral_data[PARTICIPANT]==participant) & (behavioral_data[SNIPPET]==snippet)].squeeze()
        assert(len(special_fixation.shape)==1), f'''{participant} {snippet} {special_fixation}'''
        assert(len(behavior.shape)==1)
        frp_offset = special_fixation[FIXATION_COLUMN_START]-behavior[BEHAVIORAL_COLUMN_START]
        if abs(row['Offset to ERP']-frp_offset)>=round_time_EEG(1/EEG_FREQUENCY):
            print(f'''Big delta between offset from eeg file {row['Offset to ERP']} and fixation-behavioral {frp_offset}''')
        fixation_data = pd.read_csv(get_selected_fixations_path(participant, snippet), sep=SEPARATOR, dtype={PARTICIPANT: str})
        fixation = fixation_data.reset_index().merge(special_fixation.to_frame().T, how='inner', on=FIXATION_COLUMNS)
        assert(fixation.shape[0]==1)
        fixation_index=fixation['index'].values[0]
        if fixation_index<fixation_data.index.max():
            next_fixation_start = fixation_data.loc[fixation_index+1].squeeze()[FIXATION_COLUMN_START]
        else:
            next_fixation_start = behavior[BEHAVIORAL_COLUMN_END]
        # calculate distance between fixation / behavioral and compare to given results in iteration
        # get length of fixation, and distance to next fixation
        # FIXATION_COLUMNS
        data.loc[data.shape[0]] = {
            PARTICIPANT:participant, 
            CONDITION:CONDITION_VARIANT_MATCH[get_snippet_variant(snippet)], 
            'Offset from stimulus onset to fixation onset': special_fixation[FIXATION_COLUMN_START]-behavior[BEHAVIORAL_COLUMN_START],
            'Fixation Duration': special_fixation[FIXATION_COLUMN_DURATION],
            'Duration to next fixation':(next_fixation_start-special_fixation[FIXATION_COLUMN_START]),
            'Offset to First Fixation':special_fixation[FIXATION_COLUMN_START]-fixation_data.loc[0].squeeze()[FIXATION_COLUMN_START]
        }
    data[PATH] = path
    data[SITUATION] = path.stem
    data[FIXATION_SELECTION_ALGORITHM] = frp
    data[ERP_PARAMETER_EPOCH_INTERVAL] = data.index.map(lambda i:epoch_interval)
    data.to_csv(get_erp_fixation_analysis_path(frp, snippet_group, f'{description}_statistics', 'fixation analysis'), sep=SEPARATOR)
    analyze_statistics_distribution(frp, snippet_group, description, data, 'erp frp offset2','Offset from stimulus onset to fixation onset')
    analyze_statistics_distribution(frp, snippet_group, description, data, 'fixation duration','Fixation Duration')
    analyze_statistics_distribution(frp, snippet_group, description, data, 'next fixation','Duration to next fixation')
    analyze_statistics_distribution(frp, snippet_group, description, data, '1st fixation frp offset','Offset to First Fixation')

  0%|          | 0/2 [00:00<?, ?it/s]

..\10-Data_Evaluation_Results\EEG\EEG_FRP_combs\fixation_analysis\combs_-300_1000_all_trial_erp frp offset.csv
{}
	Offset from stimulus onset to fixation onset diff_ambiguous-unambiguous might be normally distributed due to 0.255 greater equals to alpha 0.05; Result ShapiroResult(statistic=0.9487491250038147, pvalue=0.25452032685279846)
	For Offset from stimulus onset to fixation onset, the t-test paired's stance on the H1 `confusing greater than clean` indicates with alpha=0.05 and the p-value of 0.998: H0 kept, H1 not proven; Result TtestResult(statistic=-3.2183696073925603, pvalue=0.998096150209999, df=23)
	For Offset from stimulus onset to fixation onset, the t-test paired's stance on the H1 `confusing less than clean` indicates with alpha=0.05 and the p-value of 0.002: H1 accepted; Result TtestResult(statistic=-3.2183696073925603, pvalue=0.001903849790001008, df=23)
		Effect size -0.38900303234440087 small
{}
	Fixation Duration diff_ambiguous-unambiguous might be normally distribu

In [3]:
# data['Offset from stimulus onset to fixation onset (ms)'] = data['Offset from stimulus onset to fixation onset']*1000
# data['Fixation Duration (ms)'] = data['Fixation Duration']*1000
# plt.rcParams.update({'font.size': 14})
# fig, axis = plt.subplots(2, 1, figsize=(7,6))
# for i, col in enumerate(['Fixation Duration (ms)','Offset from stimulus onset to fixation onset (ms)', ]):
#     sns.violinplot(data, x=col, y=CONDITION, #hue=CONDITION, 
#                     legend=False, inner="box", cut=0, ax=axis[i], inner_kws=dict(box_width=30, whis_width=2, color=".2"))
#     axis[i].set_ylabel('')
#     axis[i].set_xlabel('Time (ms)')
#     axis[i].set_title(' ', loc='left')
# plt.tight_layout()
# plt.show()
# plt.savefig('test.png', bbox_inches='tight', pad_inches=0)
# plt.close()
